In [156]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250
print(os.getcwd())

cpu
/Users/haowa/Documents/GitHub/fcc-intro-to-llms


In [157]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
char_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [177]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
print('string_to_int')
print(string_to_int)
print('int_to_string')
print(int_to_string)

encode = lambda s: [string_to_int[c] for c in s]
#decode = lambda l: ''.join([int_to_string[i] for i in l])
def decode(l):
    decoded_string = ''
    for i in l:
        if i in int_to_string:
            decoded_string += int_to_string[i]
        else:
            print(f"Key not found: {i}")
            decoded_string += '?'  # or any placeholder for unknown tokens
    return decoded_string

#encoded_hello = encode('hello')
#decoded_hello = decode(encoded_hello)

#print(decoded_hello)

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])


string_to_int
{'\n': 0, ' ': 1, '!': 2, '"': 3, '&': 4, "'": 5, '(': 6, ')': 7, '*': 8, ',': 9, '-': 10, '.': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, ';': 23, '?': 24, 'A': 25, 'B': 26, 'C': 27, 'D': 28, 'E': 29, 'F': 30, 'G': 31, 'H': 32, 'I': 33, 'J': 34, 'K': 35, 'L': 36, 'M': 37, 'N': 38, 'O': 39, 'P': 40, 'Q': 41, 'R': 42, 'S': 43, 'T': 44, 'U': 45, 'V': 46, 'W': 47, 'X': 48, 'Y': 49, 'Z': 50, '[': 51, ']': 52, '_': 53, 'a': 54, 'b': 55, 'c': 56, 'd': 57, 'e': 58, 'f': 59, 'g': 60, 'h': 61, 'i': 62, 'j': 63, 'k': 64, 'l': 65, 'm': 66, 'n': 67, 'o': 68, 'p': 69, 'q': 70, 'r': 71, 's': 72, 't': 73, 'u': 74, 'v': 75, 'w': 76, 'x': 77, 'y': 78, 'z': 79, '\ufeff': 80}
int_to_string
{0: '\n', 1: ' ', 2: '!', 3: '"', 4: '&', 5: "'", 6: '(', 7: ')', 8: '*', 9: ',', 10: '-', 11: '.', 12: '0', 13: '1', 14: '2', 15: '3', 16: '4', 17: '5', 18: '6', 19: '7', 20: '8', 21: '9', 22: ':', 23: ';', 24: '?', 25: 'A', 26: 'B', 27: 'C', 28

In [178]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')

print('inputs:')
print(x)

print('targets:')
print(y)

tensor([106460,  62315,  47039, 153557])
inputs:
tensor([[72,  1, 66, 68, 75, 58, 57,  0],
        [55, 78,  0, 55, 71, 68, 68, 64],
        [68, 76, 58, 71,  1, 76, 68, 74],
        [68, 59,  1, 73, 61, 62, 72,  0]])
targets:
tensor([[ 1, 66, 68, 75, 58, 57,  0, 72],
        [78,  0, 55, 71, 68, 68, 64,  1],
        [76, 58, 71,  1, 76, 68, 74, 65],
        [59,  1, 73, 61, 62, 72,  0, 69]])


In [179]:
print(train_data)
print(val_data)

tensor([80,  1,  1,  ..., 71,  0, 69])
tensor([65, 54, 73,  ..., 29, 67, 57])


In [180]:
# bigram model

x = train_data[:block_size]
print(x)
y = train_data[1:block_size+1]
print(y)

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

tensor([80,  1,  1, 28, 39, 42, 39, 44])
tensor([ 1,  1, 28, 39, 42, 39, 44, 32])
when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [181]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [182]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
LLM = m.generate(context, max_new_tokens=500)[0].tolist()
print(LLM)
generated_chars = decode(LLM)
print(generated_chars)


[0, 87, 540, 479, 341, 627, 140, 105, 996, 977, 931, 502, 684, 137, 376, 401, 160, 860, 637, 819, 865, 583, 69, 181, 736, 525, 906, 675, 778, 304, 450, 682, 403, 634, 240, 573, 441, 677, 964, 174, 998, 477, 256, 101, 428, 735, 769, 278, 192, 782, 662, 180, 373, 168, 903, 554, 360, 683, 750, 348, 182, 153, 877, 594, 873, 909, 6, 773, 528, 895, 633, 568, 130, 901, 680, 359, 774, 374, 142, 914, 566, 304, 293, 990, 694, 981, 465, 289, 260, 0, 870, 191, 588, 227, 532, 579, 729, 317, 64, 682, 443, 675, 645, 279, 953, 847, 449, 455, 915, 313, 754, 414, 902, 714, 397, 38, 297, 533, 241, 522, 859, 571, 0, 956, 170, 552, 154, 7, 563, 190, 0, 204, 291, 551, 480, 735, 124, 336, 204, 133, 538, 544, 230, 254, 535, 122, 241, 443, 188, 566, 917, 479, 845, 922, 712, 291, 27, 999, 618, 968, 26, 690, 332, 258, 55, 773, 496, 551, 984, 37, 539, 145, 913, 234, 222, 294, 133, 164, 473, 411, 281, 456, 433, 263, 867, 717, 539, 477, 414, 580, 64, 82, 221, 593, 737, 155, 208, 62, 588, 710, 919, 17, 435, 775, 517

In [153]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    #if iter % eval_iters == 0:
    #    losses = estimate_loss()
    #    print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

tensor([121207,  43696,  27282, 106437])
tensor([ 76559, 157702,  92327, 101825])
tensor([128213,  81747,  78866,   8236])
tensor([111057, 177191, 154269,  95444])
tensor([128926,  37549,  65340, 137874])
tensor([133486, 123547, 116238, 167239])
tensor([ 78204,  11176, 158303,  54168])
tensor([111668, 118720,  21951, 178714])
tensor([ 64275,   2298, 157748,  83271])
tensor([ 59496, 183729,  37694,  53161])
tensor([97397, 67702, 81199, 32628])
tensor([ 71796, 160365, 121504,  66984])
tensor([116419,  64893, 100861, 182238])
tensor([ 81694, 104781, 178289,  85760])
tensor([133714, 177913,  69389,   7737])
tensor([ 77232,  72873, 106850,  53358])
tensor([172057, 158351, 173756, 145398])
tensor([155729,  89835, 148403, 150444])
tensor([156166,  93743,  46279,  68037])
tensor([ 64956, 117660, 135317, 145910])
tensor([ 29024,  95921,  51659, 130957])
tensor([ 43391,  62938,  29460, 145545])
tensor([27775, 38180, 38668, 25795])
tensor([117577, 181954,  74566, 153588])
tensor([123994, 183900, 

In [154]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


?(?g?????????????????c????????????I????????????u??8??????????????????e??????????S???????????????K?????????????????????;???????????????????'??????????????????????????????v?????????a????sx??U?????????????????u??W???????j???????a?????????????M??????????)?????????????﻿???n???m?????????H??????????????????t??????????????????????????????X???????????????????f????????????????M?????????????????u?j???????????????????????????W?02????T???M?????????????????B?Y??????????????????????????????k????????????E?????!


In [155]:
# 
a = torch.zeros(2, 3, 4)
print(a.shape)
x, y, z = a.shape
a = a.view(x,y,z)
print(a.shape)

torch.Size([2, 3, 4])
torch.Size([2, 3, 4])


***need to familiarize audience with optimizers (AdamW, Adam, SGD, MSE…) no need to jump into the formulas, just what the optimizer does for us and some of the differences/similarities between them***

1. **Mean Squared Error (MSE)**: MSE is a common loss function used in regression problems, where the goal is to predict a continuous output. It measures the average squared difference between the predicted and actual values, and is often used to train neural networks for regression tasks.
2. **Gradient Descent (GD):**  is an optimization algorithm used to minimize the loss function of a machine learning model. The loss function measures how well the model is able to predict the target variable based on the input features. The idea of GD is to iteratively adjust the model parameters in the direction of the steepest descent of the loss function
3. **Momentum**: Momentum is an extension of SGD that adds a "momentum" term to the parameter updates. This term helps smooth out the updates and allows the optimizer to continue moving in the right direction, even if the gradient changes direction or varies in magnitude. Momentum is particularly useful for training deep neural networks.
4. **RMSprop**: RMSprop is an optimization algorithm that uses a moving average of the squared gradient to adapt the learning rate of each parameter. This helps to avoid oscillations in the parameter updates and can improve convergence in some cases.
5. **Adam**: Adam is a popular optimization algorithm that combines the ideas of momentum and RMSprop. It uses a moving average of both the gradient and its squared value to adapt the learning rate of each parameter. Adam is often used as a default optimizer for deep learning models.
6. **AdamW**: AdamW is a modification of the Adam optimizer that adds weight decay to the parameter updates. This helps to regularize the model and can improve generalization performance. We will be using the AdamW optimizer as it best suits the properties of the model we will train in this video.

find more optimizers and details at torch.optim